# Pipelines:

## Import Libraries:

In [203]:
import pandas as pd

## Load Dateset:

In [204]:
df=pd.read_csv('../Data/Salary_Data.csv')
df.head()

,Age,Gender,Education Level,Job Title,Years of Experience,Salary
0,32.0,Male,Bachelor's,Software Engineer,5.0,90000.0
1,28.0,Female,Master's,Data Analyst,3.0,65000.0
2,45.0,Male,PhD,Senior Manager,15.0,150000.0
3,36.0,Female,Bachelor's,Sales Associate,7.0,60000.0
4,52.0,Male,Master's,Director,20.0,200000.0


In [205]:
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6704 entries, 0 to 6703
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Age                  6702 non-null   float64
 1   Gender               6702 non-null   object 
 2   Education Level      6701 non-null   object 
 3   Job Title            6702 non-null   object 
 4   Years of Experience  6701 non-null   float64
 5   Salary               6699 non-null   float64
dtypes: float64(3), object(3)
memory usage: 314.4+ KB


,Age,Years of Experience,Salary
count,6702.000000,6701.000000,6699.000000
mean,33.620859,8.094687,115326.964771
std,7.614633,6.059003,52786.183911
min,21.000000,0.000000,350.000000
25%,28.000000,3.000000,70000.000000
50%,32.000000,7.000000,115000.000000
75%,38.000000,12.000000,160000.000000
max,62.000000,34.000000,250000.000000


In [206]:
df = df.replace({'Bachelor\'s': 'Bachelor\'s Degree','Master\'s': 'Master\'s Degree', 'phD': 'PhD'})

In [207]:
df.isnull().sum()

Age                    2
Gender                 2
Education Level        3
Job Title              2
Years of Experience    3
Salary                 5
dtype: int64

## Make Pipelines

In [208]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [209]:
X_train,X_test,y_train,y_test=train_test_split(df.iloc[:,0:5],df['Salary'],test_size=0.2,random_state=42)

In [210]:
X_train

,Age,Gender,Education Level,Job Title,Years of Experience
2922,52.0,Female,PhD,Software Engineer Manager,29.0
4941,26.0,Male,Bachelor's Degree,Senior Product Marketing Manager,5.0
135,39.0,Female,Bachelor's Degree,Administrative Assistant,10.0
2306,23.0,Female,Bachelor's Degree,Software Engineer,1.0
3433,38.0,Male,PhD,Senior Human Resources Manager,11.0
...,...,...,...,...,...
3772,41.0,Male,PhD,Data Scientist,12.0
5191,26.0,Female,Bachelor's Degree,Social Media Manager,3.0
5226,27.0,Male,Bachelor's Degree,Product Designer,3.0
5390,31.0,Female,Bachelor's Degree,Marketing Coordinator,4.0


In [211]:
trnf1=ColumnTransformer([
    ('num',SimpleImputer(strategy='mean'),[0,4]),
    ('cat',SimpleImputer(strategy='most_frequent'),[1,2,3])
],remainder='passthrough')

In [212]:
trnf2=ColumnTransformer([
    ('ord_enco' , OrdinalEncoder(handle_unknown='use_encoded_value',unknown_value=-1,categories=[["High School", 'Bachelor\'s Degree', "Master's Degree", 'PhD']]),[2])
],remainder='passthrough')

In [213]:
trnf3 =ColumnTransformer ([
    ('ohe',OneHotEncoder(sparse_output=False, handle_unknown='ignore'), [1,3])
],remainder='passthroug')

## Create pipeline:

In [214]:
pipe=Pipeline([
    ('trnf1',trnf1),
    ('trnf2',trnf2),
    ('trnf3',trnf3),
])

In [215]:
pipe.fit(X_train)

Pipeline(steps=[('trnf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num', SimpleImputer(),
                                                  [0, 4]),
                                                 ('cat',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [1, 2, 3])])),
                ('trnf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ord_enco',
                                                  OrdinalEncoder(categories=[['High '
                                                                              'School',
                                                                              "Bachelor's "
                                                                              'Degree',
                                                                              "Master's "
                                                                              'Degree',
                                                                              'PhD']],
                                                                 handle_unknown='use_encoded_value',
                                                                 unknown_value=-1),
                                                  [2])])),
                ('trnf3',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1, 3])]))])

In [216]:
pipe=pipe.transform(X_train)